In [13]:
import spot
import buddy
from spot.jupyter import display_inline, display
from IPython.core.display import HTML
spot.setup(show_default='.a')

# Example

## Build NBA
NBA formula: $G(a \leftrightarrow b)$

In [14]:
f = spot.formula('G(a <-> b)'); nba = f.translate()

In [15]:
display_inline(nba)

In [16]:
a_varnum, b_varnum = nba.register_ap('a'),nba.register_ap('b')
print("varnum of a", a_varnum)
print("varnum of b", b_varnum)

varnum of a 0
varnum of b 1


## Create NBA Prime

In [21]:
f = spot.formula('G(a <-> b)'); nba_copy = f.translate()
prime_state = nba_copy.new_state()
nba_copy.set_init_state(prime_state)
print('Prime state', prime_state)
display_inline(nba_copy)

Prime state 1


In [26]:
tested_varnum = a_varnum
tested_state = 0

# Clean the prime state
nba_copy.kill_state(prime_state)

for edge in nba.out(tested_state):
    edge_cond = buddy.bdd_restrict(edge.cond, buddy.bdd_nithvar(tested_varnum)) & buddy.bdd_ithvar(tested_varnum)
    nba_copy.new_edge(prime_state, edge.dst, edge_cond, edge.acc)

display_inline(nba_copy)

## Is contained?

In [27]:
spot.contains(nba, nba_copy)

False

In [28]:
not spot.complement(nba).intersects(nba_copy)

False

# Unates Algorithm

## Find Unate

In [39]:
def find_unates(nba, ltl_formula, var: str, state: int):
    # Create a clone of NBA with a prime state
    nba_prime = spot.formula(ltl_formula).translate()
    prime_state = nba_prime.new_state()
    nba_prime.set_init_state(prime_state)

    # Create edges of the prime state
    tested_varnum = nba.register_ap(var)

    # Duplicate the edges
    for edge in nba.out(state):
        edge_cond = buddy.bdd_restrict(edge.cond, buddy.bdd_nithvar(tested_varnum)) & buddy.bdd_ithvar(tested_varnum)
        nba_prime.new_edge(prime_state, edge.dst, edge_cond, edge.acc)
    
    print("NBA Prime (prime state = {}): ".format(prime_state))
    display_inline(nba_prime)
    
    return spot.contains(nba, nba_prime)


In [40]:
ltl_formula = 'G(o <-> i)'
nba = spot.formula(ltl_formula).translate()

print("Original NBA: ")
display_inline(nba)

Original NBA: 


In [41]:
is_unate = find_unates(nba, ltl_formula, 'o', 0)
print('Is variable Unate? ', is_unate)

NBA Prime (prime state = 1): 


Is variable Unate?  False


## Handle Unate Variable

In [12]:
def handle_unate_var(nba, state, var):
    bdd_var = buddy.bdd_ithvar(nba.register_ap(var))
    for edge in nba.out(state):
        edge.cond = edge.cond & bdd_var

if is_unate:
    handle_unate_var(nba, 0, 'o')
display_inline(nba)